# Fake News Detection model in AWS SageMaker

## Install dependencies

In [1]:
!pip install transformers tensorflow flax tf-keras transformers[torch]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 153.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 54.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 181.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 110.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 191.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 138.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 130.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 159.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 132.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 33.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip uninstall -y torch

Found existing installation: torch 2.5.1
Uninstalling torch-2.5.1:
  Successfully uninstalled torch-2.5.1


In [3]:
!pip install torch==1.13.0 torchvision==0.14.0 torchaudio==0.13.0 --extra-index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.5/198.5 MB 45.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 69.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 62.1 MB/s eta 0:00:00


In [4]:
!pip uninstall -y tensorflow

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0


In [5]:
!pip install tensorflow

  Using cached tensorflow-2.18.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
Using cached tensorflow-2.18.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (615.3 MB)


In [6]:
import torch
print(torch.__version__)

import tensorflow as tf
print(tf.__version__)

1.13.0+cpu


2024-10-31 19:55:08.555284: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730404508.575561    7794 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730404508.581708    7794 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-31 19:55:08.601705: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.18.0


### Step 1: Load Datasets

In [7]:
import pandas as pd

# Load the datasets
fake_df = pd.read_csv('fake_news_data/Fake.csv')
real_df = pd.read_csv('fake_news_data/True.csv')

# Display the first few rows of each DataFrame to confirm loading
print(fake_df.head())
print(real_df.head())

                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  
                                               title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept t

### Step 2: Add a Label Column

In [8]:
# Add a label column
fake_df['label'] = 0  # 0 for fake news
real_df['label'] = 1   # 1 for real news

### Step 3: Combine the DataFrames

In [9]:
# Combine the two DataFrames
train_df = pd.concat([fake_df, real_df], ignore_index=True)

# Display the first few rows of the combined DataFrame
print(train_df.head())

                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  label  
0  December 31, 2017      0  
1  December 31, 2017      0  
2  December 30, 2017      0  
3  December 29, 2017      0  
4  December 25, 2017      0  


### Step 4: Prepare Labels for Encoding

In [10]:
# Prepare the labels
train_labels = train_df['label']

### Step 5: Optional - Split into Train/Test Datasets

In [11]:
from sklearn.model_selection import train_test_split

# Split into training and testing sets (you can adjust the test_size as needed)
train_df, test_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['label'])

# Prepare labels for the test set
test_labels = test_df['label']

### Step 6: Preprocess the Data for Training

#### Import the Tokenizer

In [12]:
from transformers import BertTokenizer

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

#### Tokenize the Text

In [13]:
# Tokenize the training data
train_encodings = tokenizer(list(train_df['text']), truncation=True, padding=True, max_length=512)

# Tokenize the testing data
test_encodings = tokenizer(list(test_df['text']), truncation=True, padding=True, max_length=512)

### Step 7: Prepare Data for the Model

In [14]:
import torch

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

#### Create Dataset Objects

In [15]:
train_dataset = NewsDataset(train_encodings, list(train_df['label']))
test_dataset = NewsDataset(test_encodings, list(test_df['label']))

### Step 8: Set Up the Model

#### Import and Load the Model:

In [16]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Set Training Arguments

In [20]:
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=1.5, 
    save_total_limit=2,
    save_strategy="epoch",
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=16,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_steps=1000,
    evaluation_strategy="no"
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Step 9: Train the Model

#### Initialize the Trainer: 

In [21]:
trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=test_dataset             
)

#### Train the Model:

In [22]:
last_checkpoint = "./results/checkpoint-1500"

trainer.train(resume_from_checkpoint=last_checkpoint)

Step,Training Loss
2000,0.003900
3000,0.001400


TrainOutput(global_step=3368, training_loss=0.0011594385404201698, metrics={'train_runtime': 25867.4748, 'train_samples_per_second': 2.083, 'train_steps_per_second': 0.13, 'total_flos': 1.417800232912896e+16, 'train_loss': 0.0011594385404201698, 'epoch': 1.5002227171492204})

### Step 10: Evaluate the Model

#### Define Compute Metrics Function:

In [23]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

#### Evaluate the Model:

In [24]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.0002313898439751938, 'eval_runtime': 2336.4503, 'eval_samples_per_second': 3.843, 'eval_steps_per_second': 0.241, 'epoch': 1.5002227171492204}


### Save the Model:

In [25]:
### model.save_pretrained('./fake_news_model')
tokenizer.save_pretrained('./fake_news_model')

('./fake_news_model/tokenizer_config.json',
 './fake_news_model/special_tokens_map.json',
 './fake_news_model/vocab.txt',
 './fake_news_model/added_tokens.json')

### Step 11: Test the Model with New Text

#### Tokenize the New Text:

In [ ]:
new_text = "Enter a news article here to test"
inputs = tokenizer(new_text, return_tensors="pt", truncation=True, padding=True, max_length=512)

#### Run the Model:

In [ ]:
outputs = model(**inputs)
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()
print("Fake" if predicted_class == 0 else "Real")

### Push to GitHub in SageMaker terminal:

In [ ]:
git init
git add .
git commit -m "Add fake news detection model"
git remote add origin <your-repo-url>
git push -u origin main